# nb-model_template

In [4]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_template.ipynb'   # FILL
dir_name = 'model'                  # FILL
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
import torch

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_type, pd_common_idx_rows, remove_dups_list, set_loglevel, search_df, chained_filter, get_variants, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import midx_get_level, midx_intersect, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_is_ndim
from model.common import DATASET_DIR, XG_DIR, EXPECTED_NUM_HOURS, default_dataset
from model.dataprep_util import align_first_last_cols, prune_nulls
from model.datagen_util import xgdg
from data.data_api import DataAPI
from recon.dataset_util import prep_dataset, gen_group
#from recon.split_util import get_train_test_split, gen_time_series_split, index_three_split, pd_binary_clip
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_template.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/
Using TensorFlow backend.
W0916 19:17:28.712528 139779581159232 deprecation_wrapper.py:119] From /home/kev/crunch/model/common.py:19: The name tf.train.RMSPropOptimizer is deprecated. Please use tf.compat.v1.train.RMSPropOptimizer instead.

W0916 19:17:28.713118 139779581159232 deprecation_wrapper.py:119] From /home/kev/crunch/model/common.py:19: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



## Util functions:

In [10]:
def debug_xg(xg_fname, assets=None):
    """
    Display processed data of an experiment group.
    """
    for i, (paths, recs, dfs) in enumerate(xgdg(xg_fname, delayed=True, assets=assets, filters_map=None)):
        fpath, lpath, tpath = paths
        frec, lrec, trec = recs
        logging.info('(X, y, z) -> ({fdesc}[:], {ldesc}[:], {tdesc}[:])'.format(fdesc=frec.desc, ldesc=lrec.desc, tdesc=trec.desc))
        with benchmark('time to load data') as b:
            f, l, t = dfs.compute()

        val_ratio = .2
        test_ratio = .2
        train_ratio = 1-(val_ratio+test_ratio)
        f_train_idx, f_val_idx, f_test_idx = midx_split(f.index, train_ratio, val_ratio, test_ratio)
        l_train_idx, l_val_idx, l_test_idx = midx_split(l.index, train_ratio, val_ratio, test_ratio)
        t_train_idx, t_val_idx, t_test_idx = midx_split(t.index, train_ratio, val_ratio, test_ratio)

        f_train_pd, f_val_pd, f_test_pd = f.loc[f_train_idx], f.loc[f_val_idx], f.loc[f_test_idx]
        l_train_pd, l_val_pd, l_test_pd = l.loc[l_train_idx], l.loc[l_val_idx], l.loc[l_test_idx]
        t_train_pd, t_val_pd, t_test_pd = t.loc[t_train_idx], t.loc[t_val_idx], t.loc[t_test_idx]

        if (is_type(f.index, pd.core.index.MultiIndex)):
            f_train_np, f_val_np, f_test_np = map(pd_midx_to_arr, [f_train_pd.stack(), f_val_pd.stack(), f_test_pd.stack()])
        else:
            f_train_np, f_val_np, f_test_np = f_train_pd.values, f_val_pd.values, f_test_pd.values
        l_train_np, l_val_np, l_test_np = l_train_pd.values, l_val_pd.values, l_test_pd.values
        t_train_np, t_val_np, t_test_np = t_train_pd.values, t_val_pd.values, t_test_pd.values
        
        display (i)
        display('f.shape: {}, l.shape:{}, t.shape:{}'.format(f_train_np.shape, l_train_np.shape, t_train_np.shape))
        display('f:', f_train_np)

In [11]:
debug_xg('xg0.json', assets='sp_500')

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
E0916 20:00:23.870556 139779581159232 common_util.py:2089] CRITICAL - time to load data: now


0

'f.shape: (1358, 5), l.shape:(1358,), t.shape:(1358,)'

'f:'

array([[ 902.99     ,  934.73     ,  899.35     ,  931.8      ,
         919.8747875],
       [ 929.17     ,  936.63     ,  919.53     ,  927.45     ,
         928.0481125],
       [ 931.17     ,  943.85     ,  927.28     ,  934.7      ,
         934.7745   ],
       ...,
       [1888.19     , 1896.33     , 1885.39     , 1892.49     ,
        1893.2469375],
       [1893.32     , 1901.26     , 1893.32     , 1900.53     ,
        1898.6895125],
       [1902.01     , 1912.28     , 1902.01     , 1911.91     ,
        1909.92945  ]])

...

E0916 20:00:25.316282 139779581159232 common_util.py:2089] CRITICAL - time to load data: now


39

'f.shape: (1358, 5), l.shape:(1358,), t.shape:(1358,)'

'f:'

array([[ 716.1       , 1529.6       ,  714.6       , 1529.6       ,
        1003.46666667],
       [ 817.3       , 1356.        ,  817.3       , 1356.        ,
        1085.41111111],
       [1404.8       , 1530.5       , 1383.1       , 1498.        ,
        1448.17777778],
       ...,
       [2457.5       , 2612.5       , 1946.1       , 1946.1       ,
        2283.72222222],
       [2066.7       , 2379.2       , 1844.1       , 2379.2       ,
        2172.45555556],
       [ 853.3       , 1736.2       ,  853.3       , 1736.2       ,
        1349.5       ]])